# Setup

In [3]:
# Import everything and setup things
import time
import json
import sys
import os
import datetime
import math
file_folder = os.getcwd()
if "testing_folder" in file_folder:
    file_folder = os.path.dirname(file_folder)
    sys.path.append(file_folder)
    os.chdir(file_folder)


from youtube_transcript_api import YouTubeTranscriptApi

import tiktoken
enc=tiktoken.get_encoding("cl100k_base")

keys=json.load(open("working_folder/keys.json"))

import anthropic
client = anthropic.Anthropic(api_key=keys["anthropic"])

from openai import OpenAI
os.environ["OPENAI_API_KEY"]=keys["openai"]


In [ ]:
# Cost calculation
input_cost=0
output_cost=0
total_cost=input_cost+output_cost
def get_cost(input_text,output_text):
    global input_cost
    global output_cost
    global total_cost
    input_cost+=len(enc.encode(input_text))*(3/1000000.0)
    output_cost+=len(enc.encode(output_text))*(15/1000000.0)
    total_cost=input_cost+output_cost

In [ ]:
# Get raw transcript
video_id="FC3giY7YDAQ"
raw_transcript=YouTubeTranscriptApi.get_transcript(video_id)
transcript=""
nearest_times={}
for m in raw_transcript:
    print(m['text'], m["start"])
    transcript+=str(m['text'])+"\n"
    nearest_times[len(transcript)]=m["start"]
    if len(transcript)>50000:
        break

# save as transcript.txt
with open("working_folder/transcript.txt","w") as f:
    f.write(transcript)

In [ ]:
# Get time of transcript
def get_time_at_length_transcript(length):
    i=0
    while nearest_times.get(length-i,None)==None:
        i+=1
        if (length-i)<=0:
            return list(nearest_times.values())[0]

    return(nearest_times[length-i])

# Incremental Summarization


In [ ]:
increment_chars=10000
char_start_index=0
segments=3
model_responses=[]

model_prompt="Your purpose is to take a transcript from a youtube streamer named Destiny and give a synopsis of the content and the sentiment/takes of the speaker. Include all of the topics even if they are covered briefly instead of just covering the main topic."

get_response=False

# get a certain number of segments
while (len(model_responses)<segments) and ((char_start_index+increment_chars)<=len(transcript)):
    input_transcript=transcript[char_start_index:char_start_index+increment_chars]

    conv_messages=[{"role": "user", "content": "Transcript: "+input_transcript}]
    bot_response=""

    # display start and endtime
    start_second_raw=get_time_at_length_transcript(char_start_index)
    hours = math.floor(start_second_raw / 3600)
    minutes = math.floor((start_second_raw % 3600) / 60)
    seconds = start_second_raw % 60

    # calculate end time
    end_second_raw=get_time_at_length_transcript(char_start_index+increment_chars)
    hours_end = math.floor(end_second_raw / 3600)
    minutes_end = math.floor((end_second_raw % 3600) / 60)
    seconds_end = end_second_raw % 60

    print(f"Start time {int(hours):02d}:{int(minutes):02d}:{seconds:06.3f}  End time {int(hours_end):02d}:{int(minutes_end):02d}:{seconds_end:06.3f}")

    

    if get_response:
        with client.messages.stream(
            max_tokens=2024,
            system=model_prompt,
            messages=conv_messages,
            #model="claude-3-opus-20240229",
            model="claude-3-sonnet-20240229",
        ) as stream:
            for text in stream.text_stream:
                bot_response+=text
                print(text, end="", flush=True)
        print()

        get_cost(input_transcript+model_prompt,bot_response)
        print("Cost: ",total_cost)

        model_responses.append({"bot": bot_response,"transcript": input_transcript})
    
    char_start_index+=increment_chars-300
        
